In [1]:
import numpy as np
import pandas as pd

In [2]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

--2025-06-06 12:52:50--  https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0 [following]
--2025-06-06 12:52:51--  https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc80661ddfd7f174993c5c921719.dl.dropboxusercontent.com/cd/0/inline/CrFk9ezcHov_gm0IVg2jlz8WN2EC2tl5j02WlWpLuFLtIL7YusYOw429SOJPe8J3InAdL5X_RVuCt_CTyNY9USBGYxJOQnmtMTEElDGyco1rNikDcgNbkpI0obn5dHMEn2WAyp1MFvhwKlVs2N0kmER9/file# [following]
--2025-06-06 12:52:51--  https://uc80661ddfd7f174993c5c921719.dl.dropboxusercontent.com/cd/0/in

In [5]:
# Cargar data de libros
books_df = pd.read_csv("books.csv")

# Cargar BERT large embeddings de texto
text_embeddings = np.load("goodreads_bert_large_embeddings.npy")

# Revisar formato de datos
print("Books:", books_df.shape)
print("Embeddings:", text_embeddings.shape)

Books: (4287, 24)
Embeddings: (4287, 1024)


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Crear función para recomendar libros similares en base a similitud de coseno
def recommend_books(book_index, top_k=10):
  book_vector = text_embeddings[book_index].reshape(1, -1)
  similarities = cosine_similarity(book_vector, text_embeddings).flatten()

  # Excluir propio libro
  similarities[book_index] = -1

  # Obtener top k índices
  top_indices = similarities.argsort()[::-1][:top_k]

  return books_df.iloc[top_indices][["title", "authors", "average_rating"]]

In [14]:
book_id = 42
print("Libro seleccionado:")
print(books_df.iloc[book_id][["title", "authors", "average_rating"]])

print("\Libros recomendados:")
recommend_books(book_id)

Libro seleccionado:
title             City of Bones (The Mortal Instruments, #1)
authors                                      Cassandra Clare
average_rating                                          4.12
Name: 42, dtype: object
\Libros recomendados:


,title,authors,average_rating
3344,The Mortal Instruments (The Mortal Instruments...,Cassandra Clare,4.59
234,"City of Fallen Angels (The Mortal Instruments,...",Cassandra Clare,4.16
119,"City of Ashes (The Mortal Instruments, #2)",Cassandra Clare,4.21
4019,The Last Dragonslayer (The Chronicles of Kazam...,Jasper Fforde,3.86
1126,The Mortal Instruments Boxed Set: City of Bone...,Cassandra Clare,4.57
2361,"I Hunt Killers (Jasper Dent, #1)",Barry Lyga,4.02
3359,"Private (Private, #1)",Kate Brian,3.81
659,"The Raven Boys (The Raven Cycle, #1)",Maggie Stiefvater,4.05
3816,"Welcome to the Jungle (The Dresden Files, #0.5)",Jim Butcher,4.09
3166,"Storm (Elemental, #1)",Brigid Kemmerer,3.96


In [27]:
import json

with open("goodreads_past_interactions.json") as f:
    train_interactions = json.load(f)

with open("goodreads_test_interactions.json") as f:
    test_interactions = json.load(f)

In [19]:
bookid2idx = {id_: i for i, id_ in enumerate(books_df.book_id)}
idx2bookid = {i: id_ for i, id_ in enumerate(books_df.book_id)}

In [20]:
print(idx2bookid)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 11, 10: 12, 11: 13, 12: 14, 13: 16, 14: 17, 15: 18, 16: 19, 17: 20, 18: 22, 19: 23, 20: 24, 21: 25, 22: 26, 23: 27, 24: 28, 25: 29, 26: 30, 27: 31, 28: 34, 29: 35, 30: 36, 31: 37, 32: 38, 33: 39, 34: 40, 35: 41, 36: 43, 37: 44, 38: 46, 39: 48, 40: 49, 41: 50, 42: 51, 43: 52, 44: 53, 45: 56, 46: 57, 47: 58, 48: 59, 49: 60, 50: 61, 51: 64, 52: 65, 53: 66, 54: 67, 55: 69, 56: 70, 57: 71, 58: 72, 59: 73, 60: 74, 61: 76, 62: 77, 63: 79, 64: 80, 65: 82, 66: 83, 67: 85, 68: 87, 69: 88, 70: 89, 71: 90, 72: 91, 73: 92, 74: 93, 75: 94, 76: 95, 77: 96, 78: 97, 79: 98, 80: 99, 81: 100, 82: 102, 83: 103, 84: 104, 85: 105, 86: 107, 87: 108, 88: 110, 89: 112, 90: 113, 91: 114, 92: 115, 93: 116, 94: 117, 95: 118, 96: 119, 97: 120, 98: 122, 99: 123, 100: 124, 101: 125, 102: 126, 103: 127, 104: 130, 105: 134, 106: 135, 107: 136, 108: 138, 109: 139, 110: 140, 111: 141, 112: 144, 113: 146, 114: 147, 115: 148, 116: 149, 117: 151, 118: 152, 119: 153

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

# Función que calcula la media de los embeddings de interacciones del usuario y genera recomendaciones en base a eso
def recommend_for_user(user_books, top_k=10):
    vectors = []

    for book_id in user_books:
        if book_id in bookid2idx:
            vectors.append(text_embeddings[bookid2idx[book_id]])

    if not vectors:
        return []

    user_profile = np.mean(vectors, axis=0).reshape(1, -1)
    similarities = cosine_similarity(user_profile, text_embeddings).flatten()

    # Eliminar libros con los cuales el usuario ya interactuo
    for book_id in user_books:
        if book_id in bookid2idx:
            similarities[bookid2idx[book_id]] = -1

    top_indices = similarities.argsort()[::-1][:top_k]
    return [idx2bookid[i] for i in top_indices]

In [37]:
# Métricas de evaluación
def precision_at_k(preds, truth, k):
    return len(set(preds[:k]) & set(truth)) / k

def recall_at_k(preds, truth, k):
    return len(set(preds[:k]) & set(truth)) / len(truth) if truth else 0

def ndcg_at_k(preds, truth, k):
    dcg = 0.0
    for i, p in enumerate(preds[:k]):
        if p in truth:
            dcg += 1 / np.log2(i + 2)
    idcg = sum(1 / np.log2(i + 2) for i in range(min(len(truth), k)))
    return dcg / idcg if idcg > 0 else 0

def map_at_k(preds, truth, k):
    score = 0.0
    hits = 0
    for i, p in enumerate(preds[:k]):
        if p in truth:
            hits += 1
            score += hits / (i + 1)
    return score / min(len(truth), k) if truth else 0

In [51]:
K = 10
metrics = {
    "precision": [],
    "recall": [],
    "ndcg": [],
    "map": [],
}

users_evaluated = 0

for user, past_books in train_interactions.items():
    test_books = test_interactions.get(user, [])
    if not test_books:
        continue

    recommendations = recommend_for_user(past_books, top_k=K)

    metrics["precision"].append(precision_at_k(recommendations, test_books, K))
    metrics["recall"].append(recall_at_k(recommendations, test_books, K))
    metrics["ndcg"].append(ndcg_at_k(recommendations, test_books, K))
    metrics["map"].append(map_at_k(recommendations, test_books, K))

    users_evaluated += 1

# Imprimir promedio de métricas
print(f"Se evaluaron {users_evaluated} usuarios en total.\n")
for name, scores in metrics.items():
    print(f"{name.upper()}@{K}: {np.mean(scores):.4f}")


Se evaluaron 100 usuarios en total.

PRECISION@10: 0.0060
RECALL@10: 0.0060
NDCG@10: 0.0066
MAP@10: 0.0021
